In [ ]:
import tensorflow as tf
import os
import sys

sys.path.append("../")


from collections import defaultdict
from scipy.signal import triang

from tensorflow.keras.models import load_model
from lib_dolphin.audio import *
from lib_dolphin.sequential import *
from lib_dolphin.eval import *
from lib_dolphin.discrete import *

FFT_STEP     = 128
FFT_WIN      = 512
FFT_HI       = 230
FFT_LO       = 100

D            = FFT_WIN // 2 - FFT_LO - (FFT_WIN // 2 - FFT_HI)
T            = int((RAW_AUDIO - FFT_WIN) / FFT_STEP)

MODEL_PATH = 'model_v2/'

ids           = pkl.load(open(f"{MODEL_PATH}/ids.pkl", "rb"))
inst          = pkl.load(open(f"{MODEL_PATH}/instances.pkl", "rb"))
predictions   = [x for x in pkl.load(open(f"{MODEL_PATH}/predictions.pkl", "rb"))]
lab           = pkl.load(open(f"{MODEL_PATH}/labels.pkl", "rb"))
clst          = pkl.load(open(f"{MODEL_PATH}/clusters_window.pkl", "rb"))
label_mapping = LabelMapping.mapping(clst)
dim = np.sum([c.n_clusters for c in clst.values()]) + 1
supervised    = load_model(f'{MODEL_PATH}/supervised.h5')    
encoder       = load_model(f'{MODEL_PATH}/encoder.h5')    
reverse       = {v:k for k, v in lab.items()}
encoder.summary()

In [ ]:
def spec(x):
    return spectrogram(x, FFT_LO, FFT_HI, FFT_WIN, FFT_STEP)            

In [ ]:
step =  T // 2
path = '../test_encode/larger/'
for file in os.listdir(path):
    if file.endswith('.wav'):
        print(file)
        fp = f"{path}{file}"
        x = raw(fp)
        s = spec(x)
        plottable = spectrogram(x, 0, FFT_WIN // 2, FFT_WIN, FFT_STEP)
        w = windowing(s, T)
        p = supervised.predict(w)
        e = encoder.predict(w)
        print(s.shape, w.shape, p.shape, np.stack(e).shape)
        cur = 0
        labels = []
        for j in range(p.shape[0]):
            l = np.argmax(p[j])            
            if l == 4:
                c = 0
                for i in range(0, step):
                    labels.append(c)
            else:
                x = e[j].reshape(1, e[j].shape[0])
                score = clst[l].score(x)
                cluster_number = clst[l].predict(x)[0]                                
                ci = 1 + label_mapping.fwd(l, cluster_number) 
                print(i2name(ci, reverse, label_mapping), score)
                if score > -50:
                    for i in range(0, step):                
                        labels.append(ci)
                else:
                    for i in range(0, step):
                        labels.append(0)
                    
            cur += step
        c = compress_neural(labels, len(s), reverse, label_mapping)
        plot_neural(plottable, c, None)  
        